In [1]:
path_data = '/'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
unique_topics = data_unfiltered['topics'].unique()

In [7]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [8]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [9]:
len(data)

36754

### Split topics

In [10]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [11]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [12]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [13]:
topics = dict_topic2img.keys()

In [14]:
len(topics)

270

# Create prompts for the text encoder

In [29]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [30]:
#English_topics

In [31]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [23]:
#topic2english

{'e': 'This is a photo of e'}

In [32]:
topic2english['family snapshots']

'This is a photo of family snapshots'

In [149]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


In [150]:
english_dict_topic2imgGT.keys()

dict_keys(['This is a photo of family snapshots', 'This is a photo of cutlery', 'This is a photo of family', 'This is a photo of place where eating dinner', 'This is a photo of plate of food', 'This is a photo of table with food', 'This is a photo of get water', 'This is a photo of stove/hob', 'This is a photo of light source in kitchen', 'This is a photo of drinking water', 'This is a photo of kitchen sink', 'This is a photo of bathroom privacy', 'This is a photo of backyard', 'This is a photo of front door', 'This is a photo of home', 'This is a photo of source of cool', 'This is a photo of street view', 'This is a photo of teeth', 'This is a photo of hand back', 'This is a photo of hand palm', 'This is a photo of pet', 'This is a photo of bedroom', 'This is a photo of grains', 'This is a photo of wardrobe', 'This is a photo of armchair', 'This is a photo of hallway', 'This is a photo of kitchen', 'This is a photo of books', 'This is a photo of everyday shoes', 'This is a photo of ha

## Compiling dataset

In [15]:
import pickle
with open("data/dict_genfunction_topic2imgGT.pkl","rb") as f:
    generated_functions_dict_topic2imgGT = pickle.load(f)

len(generated_functions_dict_topic2imgGT)

270

In [16]:
topics_list = []
image_ids = []
functions_list = []

for topic_, images in dict_topic2img.items():
    topics_list += [topic_] * len(list(images))
    image_ids += images
    

In [17]:
#check if both dicts align
# list out keys and values separately
a_key_list = list(dict_topic2img.keys())
a_val_list = list(dict_topic2img.values())

# list out keys and values separately
b_key_list = list(generated_functions_dict_topic2imgGT.keys())
b_val_list = list(generated_functions_dict_topic2imgGT.values())

a_val_list == b_val_list

True

In [18]:
functions_list = []
for topic_, images in generated_functions_dict_topic2imgGT.items():
    functions_list += [topic_] * len(list(images))

    
len(functions_list)

45616

In [19]:
import pickle
with open("image2Llavacaption_dict.pkl", 'rb') as f:
    image2Llavacaption_dict = pickle.load(f)

len(image2Llavacaption_dict)

36754

In [1]:
# list(image2Llavacaption_dict.keys())[0].split()

In [21]:
llava_caption_list = []
list_of_paths = [path_data + s for s in image_ids]
for image_id in list_of_paths:
    llava_cap = image2Llavacaption_dict[image_id]
    llava_caption_list.append(llava_cap.split(':')[-1].lstrip())


In [22]:
import pickle
with open("image2BLIPcaption_dict.pkl", 'rb') as f:
    image2BLIPcaption_dict = pickle.load(f)

len(image2BLIPcaption_dict)

36754

In [23]:
blip_caption_list = []
#list_of_paths = [path_data + s for s in image_ids]
for image_id in list_of_paths:
    blip_cap = image2BLIPcaption_dict[image_id]
    blip_caption_list.append(blip_cap)


In [24]:
print(f"Length of topics: {len(topics_list)}")
print(f"Length of image ids: {len(image_ids)}")
print(f"Length of generated functions: {len(functions_list)}")
print(f"Length of Llava captions: {len(llava_caption_list)}")
print(f"Length of BLIP captions: {len(blip_caption_list)}")

Length of topics: 45616
Length of image ids: 45616
Length of generated functions: 45616
Length of Llava captions: 45616
Length of BLIP captions: 45616


In [76]:
dataset_v1 = pd.DataFrame()
dataset_v1['topics'] = topics_list
dataset_v1['generated functions'] = functions_list
dataset_v1['Llava captions'] = llava_caption_list
dataset_v1['BLIP captions'] = blip_caption_list
dataset_v1['image ids'] = image_ids

dataset_v1.head()

,topics,generated functions,Llava captions,BLIP captions,image ids
0,family snapshots,This is a photo of memories of shared moments.,"The image shows a framed photo of a young man,...",my son,assets/5d4be64dcf0b3a0f3f3436d1/5d4be64dcf0b3a...
1,family snapshots,This is a photo of memories of shared moments.,"The image shows a group of people, including b...",a family with their photos,assets/5ec4f7d7f0611d7ddd74099d/5ec4f7d7f0611d...
2,family snapshots,This is a photo of memories of shared moments.,"The image shows a group of people, including b...",a group of people,assets/5d4be150cf0b3a0f3f33adab/5d4be150cf0b3a...
3,family snapshots,This is a photo of memories of shared moments.,The image shows a group of young boys standing...,a group of children,assets/5d4bee16cf0b3a0f3f350d0a/5d4bee16cf0b3a...
4,family snapshots,This is a photo of memories of shared moments.,"The image shows a group of people, including a...",a family with a man and woman and two children,assets/5ec4f54f3f62767d97a4730d/5ec4f54f3f6276...


In [77]:
dataset_v1.to_csv('data/dollar_street_reannotated_dataset_v1.csv')

In [27]:
with open ("llava_gpt4_generated_obj_dict.pkl", "rb") as f:
    llava_gpt4_generated_obj_dict = pickle.load(f)
    

len(llava_gpt4_generated_obj_dict)    

45616

In [29]:
image_ids[0]

'assets/5d4be06acf0b3a0f3f3392fa/5d4be06acf0b3a0f3f3392fa.JPG'

In [33]:
object_list = []
for funct, im_id in tqdm.tqdm(zip(functions_list, image_ids)):
    key = f"{funct}_{path_data}{im_id}"
    object_list.append(llava_gpt4_generated_obj_dict[key])
    

45616it [00:00, 1049484.77it/s]


In [35]:
len(object_list)

45616

In [36]:
dataset_v2 = pd.DataFrame()
dataset_v2['topics'] = topics_list
dataset_v2['generated functions'] = functions_list
dataset_v2['identified object'] = object_list
dataset_v2['Llava captions'] = llava_caption_list
dataset_v2['BLIP captions'] = blip_caption_list
dataset_v2['image ids'] = image_ids

dataset_v2.head()

,topics,generated functions,identified object,Llava captions,BLIP captions,image ids
0,family snapshots,This is a photo of memories of shared moments.,The image.,The image shows a man and a woman standing nex...,my father and mother,assets/5d4be06acf0b3a0f3f3392fa/5d4be06acf0b3a...
1,family snapshots,This is a photo of memories of shared moments.,Camera,"The image shows a group of people, including a...",a family posing for a photo,assets/5d4be0c4cf0b3a0f3f339d4f/5d4be0c4cf0b3a...
2,family snapshots,This is a photo of memories of shared moments.,Family photo display,"The image shows a family photo display, which ...",a family with a heart shaped frame,assets/5ec4fb23f0611d7ddd742592/5ec4fb23f0611d...
3,family snapshots,This is a photo of memories of shared moments.,Photo,The image shows a refrigerator with various it...,a couple,assets/5d4bdf89cf0b3a0f3f337d07/5d4bdf89cf0b3a...
4,family snapshots,This is a photo of memories of shared moments.,Photo.,The image shows a small child sitting in a red...,a baby sitting on a table,assets/5ec4f8e5f0611d7ddd7412be/5ec4f8e5f0611d...


In [37]:
dataset_v2.to_csv('data/dollar_street_reannotated_dataset_v2.csv')

## Add income

In [15]:
dataset_v2 = pd.read_csv('data/dollar_street_reannotated_dataset_v2.csv')

In [26]:
id_list = dataset_v2['image ids'].tolist()
income_list = [data[data['imageRelPath'] == i]['income'].item() for i in id_list]

len(income_list)
len(id_list)

45616

In [25]:
data[data['imageRelPath'] == 'assets/5d4be06acf0b3a0f3f3392fa/5d4be06acf0b3a0f3f3392fa.JPG']['income'].item()

1569.0

In [27]:
income_list[0]

1569.0

In [28]:
dataset_v2['income'] = income_list

dataset_v2.to_csv('data/dollar_street_reannotated_with_income_v2.csv')

## Add scores

In [31]:
import pickle
with open('data/dict_default_topic2score.pkl', 'rb') as f:
    dict_default_topic2score = pickle.load(f)

In [32]:
len(dict_default_topic2score)

270

In [37]:
#dict_default_topic2score['This is a photo of armchair']

In [41]:
temp_dict = {}
for key, value in tqdm.tqdm(dict_default_topic2score.items()):
    for values in value:
        topic = key.replace("This is a photo of ","")
        score = values[0]
        img_id= values[1]
    
        new_key = f"{topic}_{img_id}"
        temp_dict[new_key] = score
    

100%|██████████| 270/270 [00:09<00:00, 29.73it/s]


In [42]:
len(temp_dict)

9923580

In [45]:
score_list = []
for topic, im_id in tqdm.tqdm(zip(dataset_v2['topics'].to_list(), dataset_v2['image ids'].to_list())):
    key = f"{topic}_{im_id}"
    score_list.append(temp_dict[key])
    

45616it [00:00, 1060402.55it/s]


In [46]:
dataset_v2['scores'] = score_list

In [47]:
dataset_v2.head()

,Unnamed: 0,topics,generated functions,identified object,Llava captions,BLIP captions,image ids,income,scores
0,0,family snapshots,This is a photo of memories of shared moments.,The image.,The image shows a man and a woman standing nex...,my father and mother,assets/5d4be06acf0b3a0f3f3392fa/5d4be06acf0b3a...,1569.0,0.268664
1,1,family snapshots,This is a photo of memories of shared moments.,Camera,"The image shows a group of people, including a...",a family posing for a photo,assets/5d4be0c4cf0b3a0f3f339d4f/5d4be0c4cf0b3a...,2340.0,0.289939
2,2,family snapshots,This is a photo of memories of shared moments.,Family photo display,"The image shows a family photo display, which ...",a family with a heart shaped frame,assets/5ec4fb23f0611d7ddd742592/5ec4fb23f0611d...,1963.0,0.284776
3,3,family snapshots,This is a photo of memories of shared moments.,Photo,The image shows a refrigerator with various it...,a couple,assets/5d4bdf89cf0b3a0f3f337d07/5d4bdf89cf0b3a...,6256.0,0.267767
4,4,family snapshots,This is a photo of memories of shared moments.,Photo.,The image shows a small child sitting in a red...,a baby sitting on a table,assets/5ec4f8e5f0611d7ddd7412be/5ec4f8e5f0611d...,251.0,0.241948


In [48]:
dataset_v2.to_csv('data/dollar_street_reannotated_with_income_v2.csv')